# Randomized Benchmarking Data Analysis

In [1]:
from __future__ import print_function #python 2 & 3 compatibility
import pygsti
from pygsti.extras import rb

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Generating some fake data to analyze
To show how to do use the RB analysis, we'll generate some fake ["Direct randomized benchmarking"](https://arxiv.org/abs/1807.07975) (DRB) data. But note that the analysis is the same for DRB and Clifford RB: fitting observed average success probability (ASP) data to the exponential decay $P_m = A + Bp^m$, so it is not of much importance for this tutorial that the data is from DRB.

We'll create 5-qubit DRB data, for an (imaginary) device containing 5 qubits with ring connectivity. We'll assume that the errors consist of local depolarizing after every gate with a gate and qubit independent depolarizing rate. I.e., ....

Because these simulations are on 5 qubits, they take 

In [2]:
runsims = True

In [3]:
if runsims:
    
    nQubits = 5 
    qubit_labels = ['Q0','Q1','Q2','Q3','Q4'] 
    gate_names = ['Gi', 'Gxpi2', 'Gxmpi2', 'Gypi2', 'Gympi2', 'Gcphase'] 
    #availability = {'Gcphase':[('Q0','Q1'), ('Q1','Q2'), ('Q2','Q3'), 
    #                           ('Q3','Q0'),('Q4','Q0'), ('Q4','Q1'),
    #                           ('Q4','Q2'),('Q4','Q3')]}
    #
    availability = {'Gcphase':[('Q0','Q1'), ('Q1','Q2'), ('Q2','Q3'), 
                               ('Q3','Q4'), ('Q4','Q0')]}
    pspec = pygsti.obj.ProcessorSpec(nQubits, gate_names, availability=availability, 
                                     qubit_labels=qubit_labels)

In [4]:
if runsims: 
    
    # The local error rate of every qubit
    errorrate = 0.001
    
    # Put these into a dictionary, mapping the qubit label to the error rate (which is qubit-indep here)
    gate_errorrate_dict = {q : errorrate for q in qubit_labels}
    
    # The error type : 'uniform' means locally uniform depolarization.
    ptype = 'uniform'
    
    # This creates this error model in the format needed for the simulator
    errormodel = rb.simulate.create_locally_gate_independent_pauli_error_model(pspec, gate_errorrate_dict,
                                                                               ptype='uniform')
    
    # The DRB lengths
    lengths = [0,10,20,30,50,100,200,400]

    # The number of circuits per length
    k = 30
    
    # The counts for each circuit. Below we use a basic stochastic-unravelling simulator,
    # so the time taken is linear in the number of counts.
    counts = 50

Next, we run the simulations using `rb.simulate.rb_with_pauli_errors()`. This samples DRB circuits as specified (here using the default sampler as we leave it unspecified) and writes the data to file. This simulator is of only tangential interest to this tutorial, so is not explained any further here. See the docstrings for more information.

In [ ]:
if runsims:
    
    filename = 'tutorial_files/MySimulatedDRBData.txt'
    rbdata = rb.simulate.rb_with_pauli_errors(pspec, errormodel, lengths, k, counts, 
                                              rbtype='DRB', filename=filename, verbosity=1)

- Sampling and simulating circuit 1 of 30 at each of 8 lengths
  - Number of circuits complete = 1,2,3,4,5,6,7,8,
- Sampling and simulating circuit 2 of 30 at each of 8 lengths
  - Number of circuits complete = 1,2,3,4,5,6,7,8,
- Sampling and simulating circuit 3 of 30 at each of 8 lengths
  - Number of circuits complete = 1,2,3,4,5,6,7,8,
- Sampling and simulating circuit 4 of 30 at each of 8 lengths
  - Number of circuits complete = 1,2,3,4,5,6,7,

## Import RB summary data
Currently, `pyGSTi` only has functions for analyzing RB data that has already been summarized into a "success counts" format. So, if the data is stored in `DataSet` object, this has to be pre-analyzed...

In [ ]:
rbdata = rb.io.import_rb_summary_data('tutorial_files/MySimulatedDRBData.txt')

## Implementing the analysis
Once we have an `RBSummaryDataset` object, implement the analysis is simple. To run a "standard practice" analysis this data just needs to be passed to `rb.analysis.std_practice_analysis()`. This function implements an ordinary unweighted least-squares analysis, and estimates "error bars" on the extracted parameters using a standard non-parameteric boostrap. Note that this is *not* necessarily the best way to analyze RB data: there has recently been .... . But, we expect that this analysis method performs reasonably well in most circumstances.

This function also has some useful optional arguments, which it is sometimes important to adjust:

- .... `asymptote` ....
- `rtype`. This is...

In [ ]:
rbresults = rb.analysis.std_practice_analysis(rbdata)

And that's it.

## Looking at the results

The analysis returns an `RBResults` object, that encapsulates the analysis results and the original RB summary data.

### Plotting the data and the fit

Once we've done the analysis, we can plot the data and our fit to the data, using the `.plot()` method. By default, this plots the "full" fit, whenever this has been implemented (which it has when using `rb.analysis.std_practice_analysis`)

In [ ]:
# This requires matplotlib.
rbresults.plot()

### Getting at the estimates

In [ ]:
rbresults.fits.keys()

In [ ]:
rbresults.fits['full'].estimates

In [ ]:
rbresults.fits['full'].stds

In [ ]:
rbresults.fits['full'].bootstraps['r']

Because we are doing this analysis on simulated data, we know what the DRB error rate should be according to the theory of DRB from ["Direct randomized benchmarking for multi-qubit devices"](https://arxiv.org/abs/1807.07975).

In [ ]:
theory_predicted_r = 1 - (1-0.001)**5
observed_r = rbresults.fits['full'].estimates['r']
observed_r_std = rbresults.fits['full'].stds['r']
print("The theory-predicted r is: {}".format(theory_predicted_r))
print("The (simulated) observeed r is: {} +/- {}".format(observed_r,observed_r_std))